In [3]:
import pickle
from phystables import BasicTable, SimpleTable, make_noisy
from phystables.constants import *
import define_tables
from define_tables import WallData, GoalData, BallData, TableData
import pygame as pg
import argparse
import numpy as np


DEF_NOISE = {
"kapv": KAPV_DEF, # VonMises precision on initial velocity angle estimation
"kapb": KAPB_DEF, # VonMises precision on the bounce angle estimation
"kapm": KAPM_DEF, # VonMises precision on the motion jitter
"perr": PERR_DEF  # Gaussian error on initial position estimation
}

        
def getTable(tables_file, table_id, trial_id, noisy):
    
    with open(tables_file, 'rb') as input:
        tablesData = pickle.load(input)
    tableData = tablesData[table_id]
    
    table = BasicTable(dims=tableData.dims)
    for wall in tableData.walls:
        table.add_wall(upperleft=wall.upperleft, lowerright=wall.lowerright)

    for goal in tableData.goals:
        table.add_goal(upperleft=goal.upperleft, lowerright=goal.lowerright, onreturn=goal.onreturn, color=goal.color)
        
    balls = tableData.balls
    if trial_id == 0:
        table.add_ball(initpos=balls[0].initpos, initvel=balls[0].initvel, color=balls[0].color)
    elif trial_id == 1:
        table.add_ball(initpos=balls[1].initpos, initvel=balls[1].initvel, color=balls[1].color)
    else:
        table.add_ball(initpos=balls[0].initpos, initvel=balls[0].initvel, color=balls[0].color)
        table.add_ball(initpos=balls[1].initpos, initvel=balls[1].initvel, color=balls[1].color)
        
    if noisy:
        table = make_noisy(table, **DEF_NOISE)
    
    return table

def run_to_end(trial, max_time=10.):
    while True:
        r = trial.step(t=max_time, maxtime=max_time)
        if r is not None:
            if isinstance(r, list):
                r = r[0]
            return [r, trial.tm]
    
def run_simulations(tables_file, table_id, trial_id, noisy, num_simulations, max_time):    
    table = getTable(tables_file, table_id, trial_id, noisy)
    if noisy:
        table = make_noisy(table, **DEF_NOISE)
    simulations = []
    for i in range(num_simulations):
        ret, time = run_to_end(table, max_time)
        outcome = get_const(ret)
        ball = None
        sample = (table_id, outcome, time, ball)
        simulations.append(sample)
    return simulations

In [ ]:
!python define_tables.py

# Get Samples

In [4]:
samples = run_simulations(tables_file='tables_metadata.pkl', 
                          table_id=0, 
                          trial_id=2, 
                          noisy=True, 
                          num_simulations=10, 
                          max_time=10)
samples

[(0, 'TIMEUP', 10.000999999999896, None),
 (0, 'TIMEUP', 10.001999999999896, None),
 (0, 'TIMEUP', 10.002999999999895, None),
 (0, 'TIMEUP', 10.003999999999895, None),
 (0, 'TIMEUP', 10.004999999999894, None),
 (0, 'TIMEUP', 10.005999999999894, None),
 (0, 'TIMEUP', 10.006999999999893, None),
 (0, 'TIMEUP', 10.007999999999893, None),
 (0, 'TIMEUP', 10.008999999999892, None),
 (0, 'TIMEUP', 10.009999999999891, None)]